In [1]:
import sys, os
PATH = os.path.dirname(os.path.abspath(os.curdir))
if PATH not in sys.path:
    sys.path.insert(0, PATH)


import numpy as np
import scipy as sp
import h5py
import os

import src.Simulator as sim_system
import src.Optimizer as opt
import src.SimGrad as sim_diff


In [2]:

##! keys values are the columns of interest of exp data,
##! but the values are the ones used by the simulator 
##* the experimental data is read as it is presented, modfications are added later in the Simaulator object
##* gas species must be consistent with the chemical equations from the json file


schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "O mean":       "O",
    "N":            "N",
    "CO mean":      "CO",
    "gamma.1":      "gamma_exp"
}


###* Input files and function

reactions_file = "../reactions/reactionsCompleteV2.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.05, 'O2_F':0.05, 'CO_F':0.05, 'CO2_F':0.01, 
                    'O_S': 0.5, 'CO_S':0.1, 'Vdb_S':0.05, 'Odb_S': 0.05, 'COdb_S': 0.05}


###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO2' :  lambda const_dict, exp_dict: compute_remaining_flux(const_dict, exp_dict, 0.032),
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'O', 0.016),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

input_folder_path = "Experimental_data_Paper"

output_folder_path = "../Buffer_Data"
exp_data_file = input_folder_path + ".hdf5"

In [3]:

# data_loader = sim_data.DataLoader(schema, folder_path=input_folder_path, output_file=exp_data_file, output_folder=output_folder_path)
# data_loader.load_data(force_update=True)

In [8]:


exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)



  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_35 - 0.02*CO_F*O_S*r_40 - 0.02*CO_F*Odb_S*r_61 - 0.02*CO_F*Vdb_S*r_60 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_36*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_36*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_39 - CO_S*r_37 - CO_S*r_43 - CO_S*r_44 - CO_S*r_45 - CO_S*r_46 + r_32*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0)
  d[COdb_S]/dt = CO_F*Vdb_S*r_60 - COdb_S*O_F*r_62 - COdb_S*r_54 - COdb_S*r_55 - COdb_S*r_56 - COdb_S*r_57 - COdb_S*r_59 + Vdb_S*r_49
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_35 - 0.02*CO_S*O_F*r_39 - 0.02*COdb_S*O_F*r_62 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_34 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.

In [5]:


# exp_data_arr = sim.exp_data_arr
# reactions_list = sim.output_parser['reactions_list']


# frac_solutions_arr, gammas_result_dict_arr = sim.solve_all_conditions(exp_data_arr, reactions_list)

# # print(gammas_result_dict_arr)

# gammas_sum = np.array([sum(vec.values()) for vec in gammas_result_dict_arr])

# print(gammas_sum)



In [6]:


# frac_solutions_arr, gammas_result_arr, gamas_sum_arr = opt.solve_simulations((0.0, 1e13))


# # print(frac_solutions_arr)
# # print(gammas_result_arr)
# print(gamas_sum_arr)


In [7]:
# [0.00125796 0.0014356  0.00154653 0.0024537  0.00294202 0.00332242
#  0.00154125 0.00168082 0.00172022 0.00218509 0.00282171 0.00332337
#  0.00395099 0.00458441 0.00484624 0.00058163 0.00049913 0.00046387
#  0.00053143 0.00068255 0.00080385 0.00097211 0.00110273 0.00079454
#  0.00061466 0.00052725 0.00061035 0.00077691 0.00091746 0.00110741
#  0.0013201  0.00145621 0.00045043 0.00034063 0.00027484 0.00029706
#  0.00035499 0.00040116 0.00046218 0.00053802 0.0006347  0.00043166
#  0.0003091  0.00032839 0.00039376 0.00045037 0.00054104 0.00067284
#  0.00074725 0.00042904 0.00033168 0.0002702  0.00027877 0.00031038
#  0.00033698 0.00037796 0.00043303 0.00059027 0.00041135 0.00029744
#  0.00030642 0.00034671 0.00038439 0.00045262 0.00056855 0.00066593]